Installing Packages

In [ ]:
!pip install -r /content/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 6.5 kB/s 
     |████████████████████████████████| 1.2 MB 74.1 MB/s 
     |████████████████████████████████| 2.1 MB 63.2 MB/s 
     |████████████████████████████████| 9.3 MB 61.1 MB/s 
     |████████████████████████████████| 14.9 MB 72.7 MB/s 
     |████████████████████████████████| 7.0 MB 70.4 MB/s 
     |████████████████████████████████| 880 kB 36.0 MB/s 
     |████████████████████████████████| 3.3 MB 55.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7e23d632d25cb30c882b4a9c7f81e4909040dd69794e3b60d1b8a941231032f2
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json('pages.json', lines=True)[['url_text', 'target']]

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
le.fit(df.target)

LabelEncoder()

In [ ]:
le.classes_

array(['банкротство', 'казино', 'кредитная карта', 'мотоцикл'],
      dtype=object)

In [ ]:
df.target = le.transform(df.target)

In [ ]:
df.head()

,url_text,target
0,Купить мотоциклы - более 27 281 мотоцикла на А...,3
1,Купить новый мотоцикл в мотосалоне Байк Ленд,3
2,"Мотоцикл купить в Москве, цены, продажа, интер...",3
3,Купить мотоцикл! Цены. Фото - Продажа мото - Дром,3
4,Купить мотоциклы и мототехнику 🏍 во всех регио...,3


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=1)
train_data, valid_data = train_test_split(train_data, test_size=0.25, random_state=1)

In [ ]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

Initialize BERT classifier

In [ ]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=4,
        epochs=20,
        model_save_path='/content/bert.pt'
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Prepare data and helpers for train and evlauation

In [ ]:
classifier.preparation(
        X_train=list(train_data['url_text']),
        y_train=list(train_data['target']),
        X_valid=list(valid_data['url_text']),
        y_valid=list(valid_data['target'])
    )

Train loop

In [ ]:
classifier.train()

Epoch 1/20
Train loss 0.23683309599326302 accuracy 0.9282273603082851
Val loss 0.1372846952142658 accuracy 0.9581529581529582
----------
Epoch 2/20
Train loss 0.0925601670193665 accuracy 0.9725433526011561
Val loss 0.11814359792656472 accuracy 0.9682539682539683
----------
Epoch 3/20
Train loss 0.04894840717800738 accuracy 0.98747591522158
Val loss 0.09918023512214093 accuracy 0.9783549783549783
----------
Epoch 4/20
Train loss 0.023172833589180968 accuracy 0.9947013487475915
Val loss 0.06630253513227971 accuracy 0.9855699855699855
----------
Epoch 5/20
Train loss 0.011744325073376574 accuracy 0.9980732177263969
Val loss 0.07026107926786826 accuracy 0.9826839826839827
----------
Epoch 6/20
Train loss 0.004707548786110467 accuracy 0.9985549132947977
Val loss 0.08031102789892437 accuracy 0.9826839826839827
----------
Epoch 7/20
Train loss 0.0015400803524370284 accuracy 0.9990366088631984
Val loss 0.08052988679561895 accuracy 0.9841269841269841
----------
Epoch 8/20
Train loss 0.000917256

Check test data

In [ ]:
texts = list(test_data['url_text'])
labels = list(test_data['target'])

predictions = [classifier.predict(t) for t in texts]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.9741866404114867, recall: 0.9762830299097999, f1score: 0.975166296963948


In [ ]:
print(accuracy_score(labels, predictions))

0.9754689754689755
